In [ ]:
import pandas as pd
import numpy as np
import pyedflib
import os
import shutil
import glob
from matplotlib import pyplot as plt

In [ ]:
# Read raw PPG files

mdf = pd.read_csv('00000175-resteaze-sync.csv', sep=',')
mdf.head()

In [ ]:
df = mdf[['LedGreen', 'sleep_stage', 'sleep_state']].dropna()
df.head()

In [ ]:
df['sleep_stage'].value_counts(), df['sleep_state'].value_counts()

#### Apply band pass filter to the signal

In [ ]:
from scipy.signal import butter, sosfilt, sosfreqz

def butter_bandpass(lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog=False, btype='band', output='sos')
        return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        sos = butter_bandpass(lowcut, highcut, fs, order=order)
        y = sosfilt(sos, data)
        return y

In [ ]:
# apply bandpass filter

fs = 25
lowcut = 0.35
highcut = 5.0

pro_ppg = butter_bandpass_filter(df['LedGreen'], lowcut, highcut, fs, order=5)

In [ ]:
plt.plot(pro_ppg[800:1024], label='Filtered signal (%g Hz)' % 25)
plt.xlabel('time (ms)')
plt.grid(True)
plt.axis('tight')
plt.legend(loc='upper left')

plt.show()

In [ ]:
pro_ppg.shape[0] % 750

In [ ]:
ppg_signals = pro_ppg[:-(pro_ppg.shape[0] % 750)].reshape(-1, 750)

In [ ]:
ppg_signals.shape

In [ ]:
import datetime

datetime.datetime.fromtimestamp(int(mdf['unixTimes'][0]))

In [ ]:
timestamp = mdf['unixTimes'][0]

datetime.datetime.fromtimestamp(timestamp / 1000)

## Check EDF files for reference

In [ ]:
# Read raw and annotation from EDF files
psg_fnames = glob.glob(os.path.join("./data/sleepedf/sleep-cassette", "*PSG.edf"))
ann_fnames = glob.glob(os.path.join("./data/sleepedf/sleep-cassette", "*Hypnogram.edf"))
psg_fnames.sort()
ann_fnames.sort()
psg_fnames = np.asarray(psg_fnames)
ann_fnames = np.asarray(ann_fnames)

In [ ]:
i = 0

psg_f = pyedflib.EdfReader(psg_fnames[i])
ann_f = pyedflib.EdfReader(ann_fnames[i])

assert psg_f.getStartdatetime() == ann_f.getStartdatetime()
start_datetime = psg_f.getStartdatetime()

In [ ]:
psg_f.getFileDuration()

In [ ]:
psg_f.datarecord_duration

In [ ]:
# Extract signal from the selected channel
psg_f.getSignalLabels()

In [ ]:
psg_f.getNSamples()

In [ ]:

sampling_rate = psg_f.getSampleFrequency(0)
n_epoch_samples = int(30 * sampling_rate)
signals = psg_f.readSignal(0).reshape(-1, n_epoch_samples)

In [ ]:
sampling_rate, n_epoch_samples, psg_f.readSignal(0).shape, signals.shape

In [ ]:
signals.shape

In [ ]:
plt.plot(signals[0])

In [ ]:

ann_onsets, ann_durations, ann_stages = ann_f.readAnnotations()

In [ ]:
# Have to manually define based on the dataset
ann2label = {
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,  # Follow AASM Manual
    "Sleep stage R": 4,
    "Sleep stage ?": 6,
    "Movement time": 5
}


# Generate labels from onset and duration annotation
labels = []
total_duration = 0
ann_onsets, ann_durations, ann_stages = ann_f.readAnnotations()
for a in range(len(ann_stages)):
    onset_sec = int(ann_onsets[a])
    duration_sec = int(ann_durations[a])
    ann_str = "".join(ann_stages[a])

    # Sanity check
    assert onset_sec == total_duration

    # Get label value
    label = ann2label[ann_str]

    # Compute # of epoch for this stage
    duration_epoch = int(duration_sec / 30)

    # Generate sleep stage labels
    label_epoch = np.ones(duration_epoch, dtype=np.int) * label
    labels.append(label_epoch)

    total_duration += duration_sec

labels = np.hstack(labels)

In [ ]:

# Remove annotations that are longer than the recorded signals
labels = labels[:len(signals)]


In [ ]:
# Get epochs and their corresponding labels
x = signals.astype(np.float32)
y = labels.astype(np.int32)

In [ ]:
from sleepstage import stage_dict

# Select only sleep periods
w_edge_mins = 30
nw_idx = np.where(y != stage_dict["W"])[0]
start_idx = nw_idx[0] - (w_edge_mins * 2)
end_idx = nw_idx[-1] + (w_edge_mins * 2)
if start_idx < 0: start_idx = 0
if end_idx >= len(y): end_idx = len(y) - 1
select_idx = np.arange(start_idx, end_idx+1)
x = x[select_idx]
y = y[select_idx]
print("Shape before movement remove", x.shape, y.shape)

# Remove movement and unknown
move_idx = np.where(y == stage_dict["MOVE"])[0]
unk_idx = np.where(y == stage_dict["UNK"])[0]
if len(move_idx) > 0 or len(unk_idx) > 0:
    remove_idx = np.union1d(move_idx, unk_idx)
    print("Remove irrelavant stages")
    print("  Movement: ({}) {}".format(len(move_idx), move_idx))
    print("  Unknown: ({}) {}".format(len(unk_idx), unk_idx))
    print("  Remove: ({}) {}".format(len(remove_idx), remove_idx))
    print("  Data before removal: {}, {}".format(x.shape, y.shape))
    select_idx = np.setdiff1d(np.arange(len(x)), remove_idx)
    x = x[select_idx]
    y = y[select_idx]
    print("  Data after removal: {}, {}".format(x.shape, y.shape))

In [ ]:
# Plot the distribution of sleep stages

plt.plot(x[0])